### GAN 2D

References
- http://blog.naver.com/atelierjpro/220987973950

In [1]:
import random
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import tensorflow as tf
%matplotlib inline

# Disable GPU
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
NUM_DATA = 10 ** 4
x_real = np.column_stack((np.random.normal(4, 0.5, size=NUM_DATA), np.random.normal(2, 1.0, size=NUM_DATA)))

In [3]:
class GAN:
    def __init__(self, dim_z, dim_x):
        self.dim_x = dim_x
        self.dim_z = dim_z
        my_init=tf.truncated_normal_initializer(stddev=1.0)
        
        # Placeholders
        self.x = tf.placeholder(tf.float32, [None, self.dim_x])
        self.z = tf.placeholder(tf.float32, [None, self.dim_z])
        self.lr = tf.placeholder(tf.float32)
        
        # Generator
        with tf.variable_scope('G'):
            self.G = tf.layers.dense(self.z, self.dim_x, kernel_initializer=my_init)
        
        # Discriminator
        with tf.variable_scope('D') as scope:
            def discriminator(inputs):
                return tf.layers.dense(inputs, 1, activation=tf.sigmoid, kernel_initializer=my_init, name='D')
            self.D_real = discriminator(self.x)
            scope.reuse_variables()
            self.D_fake = discriminator(self.G)
        
        # Losses
        self.loss_G      = tf.reduce_mean(-tf.log(self.D_fake))
        self.loss_D_real = tf.reduce_mean(-tf.log(self.D_real))
        self.loss_D_fake = tf.reduce_mean(-tf.log(1 - self.D_fake))
        self.loss_D      = tf.reduce_mean((self.loss_D_real, self.loss_D_fake))
        
        # Optmizers
        def optimizer(loss, var_list):
            return tf.train.AdamOptimizer(self.lr).minimize(loss, var_list=var_list)
        g_params = [v for v in tf.trainable_variables() if v.name.startswith('G/')]
        d_params = [v for v in tf.trainable_variables() if v.name.startswith('D/')]
        self.train_G = optimizer(self.loss_G, g_params)
        self.train_D = optimizer(self.loss_D, d_params)
        
    def _generate_noise(self, cnt):
#         return np.random.normal(0, 1, size=[cnt, self.dim_z])
#         return np.random.normal(0, 1, size=[cnt, self.dim_z])
        return np.random.uniform(-1., 1., size=[cnt, self.dim_z])
    
    def train(self, sess, x, lr, train_D_only=False):
        noise = self._generate_noise(x.shape[0])
        sess.run(self.train_D, feed_dict={self.x: x, self.z: noise, self.lr: lr})
        if not train_D_only:
            sess.run(self.train_G, feed_dict={           self.z: noise, self.lr: lr})
        
    def loss(self, sess, x):
        noise = self._generate_noise(x.shape[0])
        loss_G = sess.run(self.loss_G, feed_dict={           self.z: noise})
        loss_D = sess.run(self.loss_D, feed_dict={self.x: x, self.z: noise})
        return loss_G, loss_D
    
    def generate_data(self, sess, cnt):
        noise = self._generate_noise(cnt)
        return sess.run(self.G, feed_dict={self.z: noise})

In [4]:
def plot_data(x_real, x_fake, path=None):
    plt.ioff()
    plt.rc('font', size=20)
    f, ax = plt.subplots(figsize=(10, 10))
    plt.xlabel('$x_1$')
    plt.ylabel('$x_2$')
    plt.plot(x_real[:,0], x_real[:,1], 'bo', markersize=5, markeredgewidth=0.0, alpha=0.1, label='real data')
    plt.plot(x_fake[:,0], x_fake[:,1], 'ro', markersize=5, markeredgewidth=0.0, alpha=0.1, label='generated data')
    plt.title(r'GAN for 2D Data')
    plt.axis([-6, 6, -6, 6])
    plt.grid(True)
    legend = ax.legend(loc='upper center')
    path == None and plt.show() or plt.savefig(path, bbox_inches='tight')
    plt.close()

In [5]:
tf.reset_default_graph()

with tf.Session() as sess:
    gan = GAN(dim_z=2*8, dim_x=2)
    
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    
    # Pre-train D
    MB_SIZE_PT = 2 * 8
    NUM_ITER_PT = 1000
    for iter in xrange(NUM_ITER_PT):
        x_mb = x_real[[random.randint(a=0, b=x_real.shape[0]-1) for _ in range(MB_SIZE_PT)]]
        gan.train(sess, x_mb, 0.001)
        
    # Train both D and G
    MB_SIZE = 2*5
    NUM_ITER = 100001
    num_img = 1
    for iter in xrange(NUM_ITER):
        x_mb = x_real[[random.randint(a=0, b=x_real.shape[0]-1) for _ in range(MB_SIZE)]]
        lr = 0.01 * (1 - float(iter) / NUM_ITER)
        gan.train(sess, x_mb, lr)
        
        if iter % 500 == 0:
            x_fake = gan.generate_data(sess, NUM_DATA)
            plot_data(x_real, x_fake, 'plot_gan2d/%d.png'%num_img)
            num_img += 1